In [25]:
### ================
### 在数据./git下载的数据集合/yf_amazon/ratings 抽取部分的数据来做情感分析
### [5.0, 4.0, 1.0, 2.0, 3.0] - 打分列表
### 小于3分的 - label =0
### 大于3分的 - Label =1
### 暂时先不考虑3分做为中性的问题
### 2019-11-25
### Liu,Yazhou

In [1]:
import numpy as np
import pandas as pd
import pandas as pd 
import numpy as np 
import jieba
import multiprocessing
import re
import time
import os
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from sklearn.cross_validation import train_test_split
from tensorflow.python.keras.models import Sequential,model_from_yaml
from gensim.models import KeyedVectors
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional,Dropout,Activation
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.models import save_model

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 1.在原始数据中抽取样本和标签

In [36]:
path = './main _chinese_sentiment_datas/yf_amazon/ratings/'
DataFrame = pd.read_csv(open(os.path.join(path, "ratings.csv"), 'r', 
                        encoding='utf-8'),
                        index_col=0)

### 1.1 去掉任何行或者列为空的情况

In [37]:
DataFrame = DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [ ]:
DataFrame.head()

### 1.2 将评分+title+comments 存入文件32W数据

In [ ]:
LIMITS    = 1e5*3.2  #32W 数据
ratings  = DataFrame.rating.values.tolist()
titles   = DataFrame.title.values.tolist()
comments = DataFrame.comment.values.tolist() 
productIds = DataFrame.productId.values.tolist()
writor = open('./main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titles_comments.txt','w',encoding='utf-8')
writmerge = open('./main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments.txt','w',encoding='utf-8')
for i in range(len(ratings)):
    if i<=LIMITS:
        writor.write(str(ratings[i])+'||'+titles[i]+'||'+comments[i]+'\n')
        writmerge.write(str(ratings[i])+'||'+titles[i]+','+comments[i]+'\n')#将title+comments merge一起

writor.close()
writmerge.close()

### 2.数据清洗后获得文本tokens

In [10]:
def get_cn_model(): 
    cn_model = KeyedVectors.load_word2vec_format('embeddings/sgns.zhihu.bigram', 
                                             binary=False, unicode_errors="ignore")
    
    return cn_model

def re_sub(inputs):
    ## inputs is one list and outputs is also
    texts  = inputs
    texts = re.sub("\//@[a-zA-Z\W+]+", "",texts)
    texts = re.sub("\@[a-zA-Z\w+]+", "",texts)
    texts = re.sub("[\-\#+\//@.\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、""【】~@#￥%……&*（）]+","",texts)
    texts = re.sub('[A-Za-z0-9\!\%\[\]\,\。\;\:\::\?\“\”\”“\～+\:?\;;\>>]','',texts)
    texts = re.sub('\：：?','',texts)
    texts = re.sub('\·?\－\<\·\\‘\’\?\····.?\···\×\=?','',texts)
    texts = re.sub('\－.?','',texts)
    texts = re.sub('[^0-9A-Za-z\u4e00-\u9fa5]','',texts)# 留下中文，数字，大小写
    return texts

def get_train_data(path,texts_clean,texts_target,LENS = 1):
    with open(path,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            items = line.split('||')
      
            text = items[1]
            text = re_sub(text)
            if len(text)>LENS and text!='\n':
                if eval(items[0])<3:
                    texts_target.append(0)
                    texts_clean.append(text)
                elif eval(items[0])>3:
                    texts_target.append(1)  ##### train_target 这里一定要存放数字，不能是字符串'1'/'0'
                    texts_clean.append(text)
                else:# items[0] == '3' # 中评
                    pass
                
    return texts_target,texts_clean

def get_train_tokens(train_tokens,texts_clean,cn_model,LENS =1):
    for text in texts_clean:
        cut = jieba.cut(text)
        cut_list = [i for i in cut]
        for i, word in enumerate(cut_list):
            try:
                cut_list[i] = cn_model.vocab[word].index
            except KeyError:
                cut_list[i] = 0
        train_tokens.append(cut_list)
    return train_tokens

In [11]:
cn_model = get_cn_model()

In [10]:
path = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments.txt'
texts_clean = []
texts_target= []
train_tokens = []
LENS = 2
texts_target,texts_clean = get_train_data(path,texts_clean,texts_target)

In [13]:
train_tokens = get_train_tokens(train_tokens,texts_clean,cn_model)

In [14]:
len(texts_clean),len(texts_target),len(train_tokens)

(283935, 283935, 283935)

In [11]:
texts_clean[43]

'我是这本书的亚马逊第个评论者的品质从内容设计到排版印刷均无懈可击我真喜欢我强烈推荐这本自然史'

### 2.1将清洗后的文本及评分标签记录下

In [12]:
path = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments_clean.txt'
writes = open(path,'w',encoding='utf-8')
i= -1
for line in texts_clean:
    i+=1
    writes.write(str(texts_target[i])+'||'+line+'\n')
writes.close()

### 3.计算词汇表大小和词长度

In [15]:
def max_token(train_tokens):
    num_tokens = [ len(tokens) for tokens in train_tokens ]
    num_tokens = np.array(num_tokens)
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    max_tokens = int(max_tokens)
    return max_tokens

def vocab_size(train_tokens):
    vocab_tokens = []
    i = 0
    t1 = time.time()
    for tokens in train_tokens:
        for token in tokens:
            i+=1
            if token not in vocab_tokens:
                vocab_tokens.append(token)
            else:
                pass
        if i%(1e5) == 0:
            print("处理前%d的词花费的时间是:%0.2f"%(i,(time.time() - t1)/60),'mins')
    print("词汇表大小是：",len(vocab_tokens))
    return len(vocab_tokens)

In [16]:
max_tokens = max_token(train_tokens)
max_tokens

102

In [17]:
num_words = vocab_size(train_tokens)
num_words

处理前500000的词花费的时间是:0.34 mins
处理前1200000的词花费的时间是:1.12 mins
处理前4100000的词花费的时间是:5.13 mins
处理前5800000的词花费的时间是:7.91 mins
处理前6600000的词花费的时间是:9.34 mins
词汇表大小是： 75249


75249

### 4 引入词嵌入矩阵获取特征并由样本Padding 

In [9]:
def get_embedding_matrix(embedding_dim,num_words,cn_model):
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for i in range(num_words):
        embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
    embedding_matrix = embedding_matrix.astype('float32')
    return embedding_matrix

def add_padding(train_tokens,train_target,max_tokens,num_words):
    train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')
    train_pad[train_pad>=num_words] = 0
    train_targets = np.array(train_target)
    return train_pad,train_targets
def model(y_train,y_test,X_train,X_test,epochs,batch_size,num_classes):#mask_zero = True
    model = Sequential()
    model.add(Embedding(num_words,
                        embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_tokens,
                        trainable=False))##mask_zero= True,
    model.add(Bidirectional(LSTM(units= 64, return_sequences=True)))
    model.add(LSTM(units=16, return_sequences=False))
    model.add(Dense(1, activation='sigmoid')) # Dense全连接层，输出维度是3
    #model.add(Activation('softmax'))
    optimizer = Adam(lr=1*1e-2)
    model.compile(loss='binary_crossentropy',#categorical_crossentropy
              optimizer=optimizer,
              metrics=['accuracy'])
    
    model.summary()
    path_checkpoint = './main_model/sentiment_checkpoint_Amazon2cates.keras'
    checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)
    try:
        model.load_weights(path_checkpoint)
    except Exception as e:
        print(e)
        
    earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    lr_reduction  = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)
    callbacks = [earlystopping, checkpoint,lr_reduction]
    #y_train_ = to_categorical(y_train,num_classes = num_classes)
    #y_test_  = to_categorical(y_test,num_classes = num_classes)
    
    model.fit(X_train, y_train,validation_split=0.2, epochs=epochs,
          batch_size=batch_size,
          callbacks=callbacks)
    from tensorflow.python.keras.models import save_model
    save_model(model,'./main_model/sentiment_checkpoint_Amazon2cates.h5')
    result = model.evaluate(X_test, y_test)
    print('Accuracy is :{0:.2%}'.format(result[1]))
    return result[1]*100

### 5 训练前的数据准备

In [22]:
embedding_dim = 300
num_words     = 75249
max_tokens    = 102

embedding_matrix = get_embedding_matrix(embedding_dim,num_words,cn_model)
train_pad,texts_target_pad = add_padding(train_tokens,texts_target,max_tokens,num_words)
X_train, X_test, y_train, y_test = train_test_split(train_pad, # array
                                                    texts_target,  # array
                                                    test_size=0.2,
                                                    random_state=420)

In [23]:
len(train_tokens),len(texts_target),len(train_pad)

(283935, 283935, 283935)

### 5.模型开始训练

In [24]:
epochs      = 10
batch_size  = 512*2
num_classes = 2
max_tokens  = 102
accuracy = model(y_train,y_test,X_train,X_test,epochs,batch_size,num_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 102, 300)          22574700  
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         186880    
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 22,770,877
Trainable params: 196,177
Non-trainable params: 22,574,700
_________________________________________________________________
Unable to open file (Unable to open file: name = './main_model/sentiment_checkpoint_amazon2cates.keras', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)
Train on 181718 samples, validate on 45430 samples
Epoch 1/10
181718/18

### 6 在个别新的样例上的测试效果

In [2]:
def get_cn_model(): 
    cn_model = KeyedVectors.load_word2vec_format('embeddings/sgns.zhihu.bigram', 
                                             binary=False, unicode_errors="ignore")
    
    return cn_model

def re_sub(inputs):
    ## inputs is one list and outputs is also
    texts  = inputs
    texts = re.sub("\//@[a-zA-Z\W+]+", "",texts)
    texts = re.sub("\@[a-zA-Z\w+]+", "",texts)
    texts = re.sub("[\-\#+\//@.\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、""【】~@#￥%……&*（）]+","",texts)
    texts = re.sub('[A-Za-z0-9\!\%\[\]\,\。\;\:\::\?\“\”\”“\～+\:?\;;\>>]','',texts)
    texts = re.sub('\：：?','',texts)
    texts = re.sub('\·?\－\<\·\\‘\’\?\····.?\···\×\=?','',texts)
    texts = re.sub('\－.?','',texts)
    texts = re.sub('[^0-9A-Za-z\u4e00-\u9fa5]','',texts)# 留下中文，数字，大小写
    return texts

def prediction(text,LENS =2):
    cn_model             = get_cn_model()
    text = re_sub(text)
    if len(text)>LENS and text!='\n' and text!='\t':
        cut_list = [i for i in jieba.cut(text)]
        for i, word in enumerate(cut_list):
            try:
                cut_list[i] = cn_model.vocab[word].index
                if cut_list[i]>=75249:  #nums word
                    cut_list[i] = 0
            except KeyError:
                cut_list[i] = 0
            
    #padding
    max_tokens = 102  #get from above model parameters
    train_pad = pad_sequences([cut_list], maxlen=max_tokens,
                            padding='pre', truncating='pre')
    
    # loading model
    from tensorflow.python.keras.models import load_model
    model_path = './main_model/sentiment_checkpoint_Amazon2cates.h5'
    model = load_model(model_path)
    
    result = model.predict(x=train_pad)
    
    coefs = result[0][0]
    return coefs*100  # 返回判断的阈值

### 7.取一些样本测试结果

In [4]:
text_list= [
    '垃圾完全一本垃圾也是垃圾',
    '满意产品物流也快质量不错等着宝宝出了满月就可以用了',
    '卓越快递服务太差卓越网的快递太差了服务太差了快递的宗旨不就是为了客户服务的吗',
    '硬盘太小,好像不支持win7,8系统,盒上写得兼容性只有xp,2000,nt,是不是真的?',
    '很实用,但要注意不是所有地漏都适用,家里地漏不适用,只能接别的地方了。产品设计不错,用料也可以,滤网也很有用。潜水艇的质量没的说。就是价格太贵了,所以只能三星推荐了'
]

results = []
for text in text_list:
    res = prediction(text)
    results.append(res)

In [5]:
cte = ['正面评价','中性评价','负面评价']
for i in range(len(text_list)):
    if results[i]>50:
        cte_j = 0
    elif results[i]<50:
        cte_j = 2
    else:
        cte_j = 1
    print("第%d条评价是:%s"%(i+1,text_list[i]) )
    print("模型判断这是一条: ##%s##,预测阈值是:%0.2f"%(cte[cte_j],results[i]) )
    print('\n')

第1条评价是:垃圾完全一本垃圾也是垃圾
模型判断这是一条: ##负面评价##,预测阈值是:3.94


第2条评价是:满意产品物流也快质量不错等着宝宝出了满月就可以用了
模型判断这是一条: ##正面评价##,预测阈值是:99.69


第3条评价是:卓越快递服务太差卓越网的快递太差了服务太差了快递的宗旨不就是为了客户服务的吗
模型判断这是一条: ##负面评价##,预测阈值是:32.22


第4条评价是:硬盘太小,好像不支持win7,8系统,盒上写得兼容性只有xp,2000,nt,是不是真的?
模型判断这是一条: ##负面评价##,预测阈值是:30.13


第5条评价是:很实用,但要注意不是所有地漏都适用,家里地漏不适用,只能接别的地方了。产品设计不错,用料也可以,滤网也很有用。潜水艇的质量没的说。就是价格太贵了,所以只能三星推荐了
模型判断这是一条: ##正面评价##,预测阈值是:99.72




### 7.1 取一些额外的数据作为测试集合使用 =>效果较好

In [45]:
ratings  = DataFrame.rating.values.tolist()
titles   = DataFrame.title.values.tolist()
comments = DataFrame.comment.values.tolist() 
LIMITS    = 1e5*3.2
writmerge = open('./main _chinese_sentiment_datas/yf_amazon/ratings/additional_Examples_2.txt','w',encoding='utf-8')
for i in range(len(ratings)):
    if i>LIMITS and i<LIMITS*1.5:
        writmerge.write(str(ratings[i])+'||'+titles[i]+','+comments[i]+'\n')
        
writmerge.close()

In [8]:
path = './main _chinese_sentiment_datas/yf_amazon/ratings/additional_Examples_2.txt'
texts_clean = []
texts_target= []
train_tokens = []
LENS = 2
texts_target,texts_clean = get_train_data(path,texts_clean,texts_target)

In [12]:
print("新的测试样本个数:",len(texts_target))

新的测试样本个数: 140919


In [13]:
train_tokens = get_train_tokens(train_tokens,texts_clean,cn_model)

In [14]:
embedding_dim = 300
num_words     = 75249
max_tokens    = 102

embedding_matrix = get_embedding_matrix(embedding_dim,num_words,cn_model)
train_pad,texts_target_pad = add_padding(train_tokens,texts_target,max_tokens,num_words)
X_train, X_test, y_train, y_test = train_test_split(train_pad, # array
                                                    texts_target,  # array
                                                    test_size=0.2,
                                                    random_state=420)

In [15]:
model_path = './main_model/sentiment_checkpoint_Amazon2cates.h5'
model = load_model(model_path)

In [16]:
result = model.evaluate(X_test, y_test)
print('Accuracy is :{0:.2%}'.format(result[1]))

28184/28184 [==============================] - 53s    
Accuracy is :95.69%


### 7.2加载模型后利用新的数据[11W]继续训练

In [17]:
model.fit(X_train, y_train,batch_size = 1024,validation_split = 0.2,epochs = 2)

Train on 90188 samples, validate on 22547 samples
Epoch 1/2
90188/90188 [==============================] - 591s - loss: 0.1163 - acc: 0.9605 - val_loss: 0.1184 - val_acc: 0.9602
Epoch 2/2
90188/90188 [==============================] - 696s - loss: 0.1163 - acc: 0.9605 - val_loss: 0.1184 - val_acc: 0.9602


In [19]:
result = model.evaluate(X_test, y_test)
print('New accuracy is :{0:.2%}'.format(result[1]))

28184/28184 [==============================] - 59s    

New accuracy is :95.69%


In [18]:
from tensorflow.python.keras.models import save_model
save_model(model,'./main_model/sentiment_checkpoint_Amazon2cates.h5')